In [3]:
import networkx as nx
from infomap import Infomap

twitter: nx.DiGraph = nx.read_edgelist("sampled_twitter_graph3.txt", create_using=nx.DiGraph)

twitter.number_of_nodes(), twitter.number_of_edges(), twitter.is_directed()


(0, 0, True)

In [6]:

im = Infomap("--directed")
print("Infomap にリンクを追加しています...")
im.add_networkx_graph(twitter)
im.run()

print(f"検出されたコミュニティ数 (Infomap): {im.num_top_modules}")

communities = {}
communities = {}
for node in im.nodes:
    if node.is_leaf: 
        module_id = node.module_id
        
        # --- 修正点 ---
        # node.node_name ではなく、
        # node.node_id (内部ID) を im.get_name() に渡して
        # 元の文字列ノード名を取得します。
        node_name = im.get_name(node.node_id)
        
        if module_id not in communities:
            communities[module_id] = set()
        communities[module_id].add(node_name)
        
twitter_comms_infomap = list(communities.values())
print(f"検出されたコミュニティ数 (リスト化後): {len(twitter_comms_infomap)}")
#print(f"モジュラリティ: {communities.modularity:.4f}")

Infomap にリンクを追加しています...
  Infomap v2.8.0 starts at 検出されたコミュニティ数 (Infomap): 2
2025-11-06 15:50:55
  -> Input network: 
  -> No file output!
  -> Configuration: directed
  -> Ordinary network input, using the Map Equation for first order network flows
Calculating global network flow using flow model 'directed'... 
  -> Using unrecorded teleportation to links. 
  -> PageRank calculation done in 200 iterations.

  => Sum node flow: 1, sum link flow: 1
Build internal network with 20023 nodes and 20021 links...
  -> One-level codelength: 0.308873873

Trial 1/1 starting at 2025-11-06 15:50:55
Two-level compression: 1e+02% 0% 
Partitioned to codelength 8.67361738e-19 + 0 = 8.67361738e-19 in 2 modules.
Super-level compression: to codelength 8.67361738e-19 in 2 top modules.

Recursive sub-structure compression: 0% . Found 2 levels with codelength 8.67361738e-19

=> Trial 1/1 finished in 0.057977042s with codelength 8.67361738e-19


Summary after 1 trial
Best end modular solution in 2 levels:
Per

In [11]:
import os
import json # JSONライブラリを使用
import numpy as np
# (networkx as nx もインポートされている前提)

# --- 1. 保存用フォルダの作成 ---
output_folder = "adj_json_communities_txt2" # 保存先のフォルダ名
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"\n保存用フォルダ '{output_folder}' を作成しました。")
else:
    print(f"\n保存用フォルダ '{output_folder}' は既に存在します。")


# --- 2. 各コミュニティを指定の形式で .txt ファイルに保存 ---
print("各コミュニティを隣接行列の辞書形式で、txtファイルに保存しています...")

# twitter_comms が定義されている前提でループを開始
for i, community_nodes in enumerate(twitter_comms2):
    if len(community_nodes) < 2:
        continue
    
    # Step 2-1: サブグラフを生成
    subgraph = twitter.subgraph(community_nodes)
    
    # Step 2-2: サブグラフから隣接行列を生成
    node_list = list(subgraph.nodes())
    adj_matrix_np = nx.to_numpy_array(subgraph, nodelist=node_list, dtype=int)
    
    # Step 2-3: NumPy配列をご希望の辞書形式に変換
    adjacency_dict = {}
    for row_idx, source_node in enumerate(node_list):
        row_dict = {}
        for col_idx, target_node in enumerate(node_list):
            row_dict[target_node] = int(adj_matrix_np[row_idx, col_idx])
        
        adjacency_dict[source_node] = row_dict
        
    # Step 2-4: ファイルパスを定義 (★拡張子を .txt に)
    file_path = os.path.join(output_folder, f"community_{i}.txt")
    
    # Step 2-5: TXTファイルとして書き出す (★json.dumpを使用)
    with open(file_path, 'w', encoding='utf-8') as f:
        # json.dumpで書き出すと、ご提示の例の形式と一致します
        # indent=2 を指定すると、例のように整形されて見やすくなります
        json.dump(adjacency_dict, f, indent=2) 

print(f"\n処理完了: {len(twitter_comms2)}個のコミュニティを '{output_folder}' フォルダにTXTファイルとして保存しました。")


保存用フォルダ 'adj_json_communities_txt2' を作成しました。
各コミュニティを隣接行列の辞書形式で、txtファイルに保存しています...

処理完了: 3個のコミュニティを 'adj_json_communities_txt2' フォルダにTXTファイルとして保存しました。
